In [ ]:
from tkinter import *
import numpy
import openpyxl
from scipy.optimize import fsolve

In [3]:
excel_document = openpyxl.load_workbook('Tabla_coeficientes.xlsx')
#Coeficientes = excel_document.get_sheet_by_name('Hoja1')
Coeficientes_ternas = excel_document['Hoja1']

#print (Coeficientes['A2'].value)

#función que entrega una lista con la mezcla simplificada
gases=['methane','ethane','propane','batadiene','butylene','propane','propylene','hexanes','nitrogen','carbonmonoxide','hydrogen']
mezcla_gases_inertes_mas_methano=[ 90.0900,1.46,1.04]#composición sin normalizar del methano, dioxido de carbono y nitrogeno, respectivamente

def simplificar(tipos,composicion): #tipos son los tipos de gases existentes, y composicion son los valores de entrada
#     lista_gases=[]
#     for i in range(len(gaslist)):
#         lista_gases.append([tipos[i],composicion[i],i])
    lista_simplificada=[]
    methane=composicion[0]
    ethane=composicion[1]
    propane=composicion[2]
    butane = composicion[3] + composicion[4] + 2.3*(composicion[5]+composicion[6]) + 5.3*composicion[7]
    hydrogen=composicion[10]
    lista=[methane,ethane,propane,butane,hydrogen]
    gases=['methane','ethane','propane','butane','hydrogen']
    def f(x):
        return 100-x*lista[0]-x*lista[1]-x*lista[2]-x*lista[3]-x*lista[4]
    n=float(fsolve(f,50))
    for s in range(0,len(lista)):
        numero=lista[s]
        lista[s]=round(numero*n,4)
#   lista_simplificada=[['methane',lista[0]],['ethane',lista[1]],['propane',lista[2]],['butane',lista[3]],['hydrogen',lista[4]]]
    for i in range(len(lista)):
        if lista[i]!=0:
            lista_simplificada.append([gases[i],lista[i]])
    mezcla_gases_inertes_mas_methano=[composicion[0],composicion[9],composicion[8]]
    return lista_simplificada,mezcla_gases_inertes_mas_methano

#lo que está comentado adentro es por si es que se necesita saber cual era la mezcla original, con nombre

#sale así
#mezcla_simplificada=[['methane', 92.0460], ['ethane', 5.6603], ['propane', 1.3487], ['butane', 0.9451]]

##########################################
#selección de ternas revisando en este orden

#1) Carbon Monoxide
#2) Butadiene
#3) Butylene
#4) Ethylene
#5) Propylene
#6) Hydrogen Sulphide
#7) Hydrogen
#8) Propane
#9) Ethane
#10) Butane
#11) Methane

#representado en esta lista

prioridad=(['carbon Monoxide', 'butadiene',  
           'butylene', 'ethylene', 'propylene', 
           'hydrogen sulphide', 'hydrogen', 'propane', 'ethane', 
          'butane','methane'])

#paso los datos del excel a una lista de listas,"datos_ternas", con la función extraccion_base_datos
#cada elemento de la lista "datos_ternas" tiene los siguientes elementos:
#posición[0]=nombre terna (ejemplo "A2")
#posición[1], [2] y [3]= elementos "x", "y" y "z" de la terna (ejemplo: x='Propane',y='Ethane',z='Butane')
#posición [4] a la [23]= coeficientes a(numero1,numero2) de la terna
#posición [24], [25]= límite máximo y mínimo, en porcentaje de volumen, del elemento "x" en la terna
#posición [26], [27]= límite máximo y mínimo, en porcentaje de volumen, del elemento "y" en la terna
#posición [28], [29]= límite máximo y mínimo, en porcentaje de volumen, del elemento "y" en la terna

#if Datos_ternas[11][3]==None:
 #   print ('no existe oe')

#esta función extrae los datos del excel, dejandolos con la forma descrita arriba
def extraccion_base_datos(data):
    Datos_ternas=[]
    letra=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T']

    for i in range(1,20):
        terna=[]
        for j in range (1, 31):
            terna.append(data[letra[i]+str(j)].value)
        #print (terna)
        Datos_ternas.append(terna)      
    return Datos_ternas

#esta función ordena la mezcla simplificada por prioridad, para recorrerla luego

def orden_busqueda(mezcla, prioridad):
    mezcla_ordenada=[]
    for i in range(0, len(prioridad)):
        for j in range(0, len(mezcla)):
            if mezcla[j][0]==prioridad[i]:
                mezcla_ordenada.append(mezcla[j])
    return mezcla_ordenada

#suma los elementos de una lista
def sumalista(listaNumeros):
    laSuma = 0
    for i in listaNumeros:
        laSuma = laSuma + i
    return laSuma

#calcula "V sum" de cada componente (sirve para la función de cálculo de fitness number)
def vsum(componente, data):
    suma=[]
    for i in range(0, len(data)):
        if data[i][1]==componente:
            suma.append(min(100,data[i][24] +15))
        elif data[i][2]==componente:
            suma.append(min(100,data[i][26] +15))
        elif data[i][3]==componente:
            suma.append(min(100,data[i][28] +15))
        else:
            suma.append(0)
    #print (suma)
    return sumalista(suma)
    
#calcula el fitness number de una terna
def fitness_calculus(terna, mezcla, data):
    suma=[]
    for i in range(0,len(mezcla)):
        if terna[1]==mezcla[i][0]:
            A=((min(100,terna[24] +15))*mezcla[i][1])/vsum(mezcla[i][0], data)
            suma.append(A)
        elif terna[2]==mezcla[i][0]:
            A=((min(100,terna[26] +15))*mezcla[i][1])/vsum(mezcla[i][0], data)
            suma.append(A)
        elif terna[3]==mezcla[i][0]:
            A=((min(100,terna[28] +15))*mezcla[i][1])/vsum(mezcla[i][0], data)
            suma.append(A)
        else:
            suma.append(0)    
    return  sumalista(suma)


#Hace una nueva base de datos (temporal), en la que agrega a la posición (30), el fittness number de la terna respecto
#de la mezcla
def agregar_fitness_number(mezcla, data):
    datan=data.copy()
    for i in range(0,len(datan)):
        datan[i].append(fitness_calculus(data[i], mezcla, data))
    return datan

#busca las ternas que tengan los tres componentes en la mezcla simlificada

#se me ocurrió que esto podría entregar una lista con todos los elementos que tienen sus tres componentes 
#representados, otra con dos...

#print (Coeficientes['A2'].value)

#entrega una lista de listas:
#cada componente de la lista tiene:

def clasificacion_por_elementos_componentes(mezcla, data): 
    #ordena las ternas en:
    #una lista de ternas que tienen sus tres componentes en la mezcla
    #una lista de ternas que tienen 2 componentes en la mezcla
    #una lista de ternas que tienen 1 componente en la mezcla
    #una lista de ternas que tienen 0 componentes en la mezcla
    #cada elemento de las listas está compuesta de [terna, número de elementos representados en la mezcla, fit number]
    #estas listas están, a su vez, en una lista más grande
    lista3=[]
    lista2=[]
    lista1=[]
    lista0=[]
    
    for i in range(0, len(data)):
        cont=0
        for j in range(0,len(mezcla)):
            for z in range(1,4):
                if data[i][z] == mezcla[j][0]:
                    cont+=1
        if cont==3:
            lista3.append([data[i], cont, data[i][30]])
        if cont==2:
            lista2.append([data[i], cont, data[i][30]])
        if cont==1:
            lista1.append([data[i], cont, data[i][30]])
        if cont==0:
            lista0.append([data[i], cont, data[i][30]])
        #lista.append([data[i][0], cont, data[i][30]])
    lista_total=[lista3, lista2, lista2, lista0]
    return lista_total
    #l=sorted(lista, key=lambda tup:(-tup[1], -tup[2]))  #qué es esto -> esto ordena la lista primero el componente [1] y luego el [2]

#hace una lista de ceros(lo necesito XD)
def zerolistmaker(n):
    listofzeros = [0] * n
    return listofzeros


def seleccion_ternas(mezcla, represent): #función que selecciona las ternas
    contadores_representacion=zerolistmaker(len(mezcla)) #cuenta cuantas veces esta representado cada elemento en las ternas
    ternas_elegidas=[] #aquí almacena las terna a medida que las elige
    parametro=True
    while parametro==True:
        conti=0
        for zz in range(0,len(contadores_representacion)):
            if contadores_representacion[zz]>=2:
                conti=conti+1
        if conti==len(contadores_representacion):
            parametro=False
            break            
        for i in range(0, len(mezcla)): #Recorre los componentes de la mezcla
            if contadores_representacion[i]<2: #para ver que los componentes no estén presentados más de dos veces
                max_fit=0 #para poder elegir la terna con mayor fit_number
                for j in range(0, len(represent)): #recorre las listas de la función #clasificacion_por_elementos_componentes_osses 

                    for k in range(0,len(represent[j])): #recorre todas las ternas con un mismo contador

                        if mezcla[i][0]==represent[j][k][0][1] or mezcla[i][0]==represent[j][k][0][2] or mezcla[i][0]==represent[j][k][0][3]:
                            #El 'if ' de arriba busca ternas el las que el componente  i de la mezcla esté presente
                            if represent[j][k][0] not in ternas_elegidas: #evita elegir la misma terna dos veces... o tres...
                                if represent[j][k][2]>max_fit: #busca la terna con mayor fitt number
                                    terna_elegir=represent[j][k][0]
                                    max_fit=represent[j][k][2]

                ternas_elegidas.append(terna_elegir) 
                for s in range(0, len(mezcla)): #una vez elegida la terna, suma los contadores
                    if mezcla[s][0] in terna_elegir[1] or mezcla[s][0] in terna_elegir[2] or mezcla[s][0] in terna_elegir[3]:
                        contadores_representacion[s]=contadores_representacion[s]+1
    return ternas_elegidas

def dividir_comonentes_mezcla_en_las_ternas(ternas_elegidas, mezcla):
    fraccion_cada_elemento=zerolistmaker(len(mezcla))
    for i in range(0,len(mezcla)):
        cont=0
        for j in range(0, len(ternas_elegidas)):
            if mezcla[i][0]==ternas_elegidas[j][1] or mezcla[i][0]==ternas_elegidas[j][2] or mezcla[i][0]==ternas_elegidas[j][3]:
                cont=cont+1
        fraccion=mezcla[i][1]/cont
        fraccion_cada_elemento[i]=fraccion
        
    return fraccion_cada_elemento

def Fraccion_mezcla_cada_elemento_terna(ternas_elegidas,mezcla, fraccion):
    #entrega una lista de la siguente forma:
    #[terna, [fraccionVelemento1, fraccionVelemento2, fraccionVelemento3], ponderación en el total]
    ternas_elegidas_mas_fraccion_volumen=[]
    for i in range(0, len(ternas_elegidas)):
        fraccion_volumen=[0,0,0]
        for j in range(0,len(mezcla)):
            if mezcla[j][0]==ternas_elegidas[i][1]:
                fraccion_volumen[0]=fraccion[j]
            if mezcla[j][0]==ternas_elegidas[i][2]:
                fraccion_volumen[1]=fraccion[j]
            if mezcla[j][0]==ternas_elegidas[i][3]:
                fraccion_volumen[2]=fraccion[j]
        #print (fraccion_volumen)
        peso_en_el_total=sumalista(fraccion_volumen)/100.
        def f(x):
            return 100-x*fraccion_volumen[0]-x*fraccion_volumen[1]-x*fraccion_volumen[2]
        n=float(fsolve(f,50))
        #print (n)
        for s in range(0,len(fraccion_volumen)):
            numero=fraccion_volumen[s]
            fraccion_volumen[s]=numero*n
        #print (fraccion_volumen)
        ternas_elegidas_mas_fraccion_volumen.append([ternas_elegidas[i], fraccion_volumen, peso_en_el_total])
    return ternas_elegidas_mas_fraccion_volumen

def calculo_numero_metano_terna(ternax):
    MN=0
    terna=ternax[0]
    fr=ternax[1]
    def aux(i,j,fraccion):
        return ((fraccion[0])**i)*((fraccion[1])**j)
    #i=0, 1, 2,....
    aux0=terna[4]*aux(0,0,fr)+terna[6]*aux(0,1,fr)+terna[9]*aux(0,2,fr)+terna[13]*aux(0,3,fr)+terna[18]*aux(0,4,fr)+terna[22]*aux(0,5,fr)+terna[23]*aux(0,6,fr)
    aux1=terna[5]*aux(1,0,fr)+terna[8]*aux(1,1,fr)+terna[12]*aux(1,2,fr)+terna[17]*aux(1,3,fr)
    aux2=terna[7]*aux(2,0,fr)+terna[11]*aux(2,1,fr)+terna[16]*aux(2,2,fr)
    aux3=terna[10]*aux(3,0,fr)+terna[15]*aux(3,1,fr)
    aux4=terna[14]*aux(4,0,fr)
    aux5=terna[19]*aux(5,0,fr)
    aux6=terna[20]*aux(6,0,fr)
    aux7=terna[21]*aux(7,0,fr)  
    MN_terna=aux0+aux1+aux2+aux3+aux4+aux5+aux6+aux7
    return MN_terna

############
#falta una función que minimize la iferencia entre el número de methano mayor y el menor###############################
############


def MN_sin_considerar_gases_inertes(ternas_elegidas_mas_fraccion_volumen):
    MN=0
    for i in range(0,len(ternas_elegidas_mas_fraccion_volumen)):
        aux=calculo_numero_metano_terna(ternas_elegidas_mas_fraccion_volumen[i])*ternas_elegidas_mas_fraccion_volumen[i][2]
        MN=aux+MN
    return MN

def MN_de_los_gases_inertes(Mezcla_gases_inertes, ternas):
    def f(x): #normalización de la mezcla
            return 100-x*Mezcla_gases_inertes[0]-x*Mezcla_gases_inertes[1]-x*Mezcla_gases_inertes[2]
    n=float(fsolve(f,50))
    for i in range(0,len(Mezcla_gases_inertes)):
        numero=Mezcla_gases_inertes[i]
        Mezcla_gases_inertes[i]=numero*n
    terna=[ternas[18], Mezcla_gases_inertes, 100.0]
    MNinert=calculo_numero_metano_terna(terna)
    return MNinert
        

NameError: name 'openpyxl' is not defined

In [2]:
#mezcla_simplificada=simplificar(gases,gaslist) #gaslist es la lista que entra en el programa
mezcla_simplificada=[['methane', 92.0460], ['ethane', 5.6603], ['propane', 1.3487], ['butane', 0.9451]]
Datos_ternas=extraccion_base_datos(Coeficientes_ternas) #Datos extraidos de excel
mezcla_ordenada=orden_busqueda(mezcla_simplificada, prioridad) #Mezcla ordenada por prioridad

Data_fitness=agregar_fitness_number(mezcla_ordenada, Datos_ternas)#Agrega fitnes number a la base de datos, posición [30]

#Primer_ordenamiento1=clasificacion_por_elementos_componentes(mezcla_ordenada, Data_fitness)
Primer_ordenamiento=clasificacion_por_elementos_componentes(mezcla_ordenada, Data_fitness)
#print (Primer_ordenamiento)
#print (mezcla_ordenada)
ternas_elegidas=seleccion_ternas(mezcla_ordenada, Primer_ordenamiento) #selección de las ternas necesarias
fraccion_cada_elemento_en_las_ternas=dividir_comonentes_mezcla_en_las_ternas(ternas_elegidas, mezcla_ordenada)#divide la fraccion de volumen de cada componente de la mezcla en el número de ternas en el que está
#print (fraccion_cada_elemento_en_las_ternas)
ff=Fraccion_mezcla_cada_elemento_terna(ternas_elegidas,mezcla_ordenada, fraccion_cada_elemento_en_las_ternas)
MN1=MN_sin_considerar_gases_inertes(ff)
MN1
MNinert=MN_de_los_gases_inertes(mezcla_gases_inertes_mas_methano,Datos_ternas)
#print (MNinert)
MN=int(MN1+MNinert-100.00)
print (MN)

print (calculo_numero_metano_terna(ff[0]))
print (calculo_numero_metano_terna(ff[1]))
print (calculo_numero_metano_terna(ff[2]))
#ff[0]



NameError: name 'extraccion_base_datos' is not defined

In [6]:
#"Constantes"
root = Tk()
root.title("Methane Number")
root.geometry("500x400")
root["bg"]="#87cefa"
SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
methane = DoubleVar()
ethane = DoubleVar()
propane = DoubleVar()
ibutane = DoubleVar()
nbutane = DoubleVar()
ipentane = DoubleVar()
npentane = DoubleVar()
hexanes = DoubleVar()
#C6H14 = DoubleVar()
#C7H16 = DoubleVar()
#C8H18 = DoubleVar()
#C9H20 = DoubleVar()
#C10H22 = DoubleVar()
nitrogen = DoubleVar()
#CO = DoubleVar()
carbondioxide = DoubleVar()
hydrogen = DoubleVar()
#O2 = DoubleVar()


#Añadir Funciones
def getResultforMN():
    gaslist = [methane.get(),ethane.get(),propane.get(),ibutane.get(),nbutane.get(),ipentane.get(),npentane.get(),hexanes.get(),nitrogen.get(),carbondioxide.get(),hydrogen.get()]
    suma=sum(gaslist)
    if 0<suma<=100:
        mezcla_simplificada,mezcla_gases_inertes_mas_methano=simplificar(gases,gaslist)
        print(mezcla_simplificada)
        print(mezcla_gases_inertes_mas_methano)
        #mezcla_simplificada=[['methane', 92.0460], ['ethane', 5.6603], ['propane', 1.3487], ['butane', 0.9451]]
        Datos_ternas=extraccion_base_datos(Coeficientes_ternas) #Datos extraidos de excel
        mezcla_ordenada=orden_busqueda(mezcla_simplificada, prioridad) #Mezcla ordenada por prioridad
        Data_fitness=agregar_fitness_number(mezcla_ordenada, Datos_ternas)#Agrega fitnes number a la base de datos, posición [30]
        Primer_ordenamiento=clasificacion_por_elementos_componentes(mezcla_ordenada, Data_fitness)
        ternas_elegidas=seleccion_ternas(mezcla_ordenada, Primer_ordenamiento) #selección de las ternas necesarias
        print(ternas_elegidas)
        fraccion_cada_elemento_en_las_ternas=dividir_comonentes_mezcla_en_las_ternas(ternas_elegidas, mezcla_ordenada)#divide la fraccion de volumen de cada componente de la mezcla en el número de ternas en el que está
        ff=Fraccion_mezcla_cada_elemento_terna(ternas_elegidas,mezcla_ordenada, fraccion_cada_elemento_en_las_ternas)
        
        MN1=MN_sin_considerar_gases_inertes(ff)
        MN1
        MNinert=MN_de_los_gases_inertes(mezcla_gases_inertes_mas_methano,Datos_ternas)
        MN=int(MN1+MNinert-100.00)
        resultado.set(MN)
    else:   
        resultado.set("Off the range, update the mixture!")

    return 

#Añadir mas parametros para GUI 
#METANO
Methane = Label(root, text="Methane, CH4".translate(SUB), fg="Black", bg="#87cefa").place(x=10,y=10)
intCH4 = Entry(root, textvariable=methane).place(x=130,y=10)

#ETANO
Ethane = Label(root, text="Ethane, C2H6".translate(SUB), fg="Black", bg="#87cefa").place(x=10,y=30)
intC2H6 = Entry(root, textvariable=ethane).place(x=130,y=30)

#PROPANO
Propane = Label(root, text="Propane, C3H8".translate(SUB), fg="Black", bg="#87cefa").place(x=10,y=50)
intC3H8 = Entry(root, textvariable=propane).place(x=130,y=50)

#i-BUTANO
iButano = Label(root, text="i-Butane, i-C4H10".translate(SUB), fg="Black", bg="#87cefa").place(x=10,y=70)
intIC4H10 = Entry(root, textvariable=ibutane).place(x=130,y=70)

#n-Butano
nButano = Label(root, text="n-Butane, n-C4H10".translate(SUB), fg="Black", bg="#87cefa").place(x=10,y=90)
intNC4H10 = Entry(root, textvariable=nbutane).place(x=130,y=90)

#i-Pentano
iPentano = Label(root, text="i-Pentane, i-C5H12".translate(SUB), fg="Black", bg="#87cefa").place(x=10,y=110)
intIC5H12 = Entry(root, textvariable=ipentane).place(x=130,y=110)

# n-Pentano
nPentano = Label(root, text="n-Pentane, n-C5H12".translate(SUB), fg="Black", bg="#87cefa").place(x=10,y=130)
intNC5H12 = Entry(root, textvariable=npentane).place(x=130,y=130)

#Hexanes+
Hexane = Label(root, text="Hexanes+".translate(SUB), fg="Black", bg="#87cefa").place(x=10,y=150)
intC6H14 = Entry(root, textvariable=hexanes).place(x=130,y=150)

#Nitrogen
Nitrogen = Label(root, text="Nitrogen, N2".translate(SUB), fg="Black", bg="#87cefa").place(x=10,y=170)
intN2 = Entry(root, textvariable=nitrogen).place(x=130,y=170)

#Carbon Dioxide
CarbonDioxide = Label(root, text="Carbon Dioxide, CO2".translate(SUB), fg="Black", bg="#87cefa").place(x=10,y=190)
intCO2 = Entry(root, textvariable=carbondioxide).place(x=130,y=190)

#Hydrogen
Hydrogen = Label(root, text="Hydrogen, H2".translate(SUB), fg="Black", bg="#87cefa").place(x=10,y=210)
intH2 = Entry(root, textvariable=hydrogen).place(x=130,y=210)

#Resultado
resultado=StringVar()
resultados = Label(root, text="Número de Metano".translate(SUB), fg="Black", bg="#87cefa").place(x=10,y=250)
numero = Label(root, textvariable=resultado,bg="#87cefa").place(x=130,y=250)
resultado.set("")

calculate = Button(root, text="Compute MN!", fg="Blue", command=getResultforMN).place(x=400,y=320)

close = Button(root, text="Exit Program.", fg="Red", command=root.quit).place(x=400,y=350)

root.mainloop()
#Nombre Gas ;   Variable  ; Unit
#Methane    ;   CH4       ; % mole
#Ethane     ;   C2H6      ; % mole
#Propane    ;   C3H8      ; % mole
#i-Butane   ;   IC4H10    ; % mole
#n-Butane   ;   NC4H10    ; % mole
#i-Pentane  ;   IC5H12    ; % mole
#n-Pentane  ;   NC5H12    ; % mole
#hexanes+   ;             ; % mole
#Nitrogeno  ;   N2        ; % mole
#Diox Carb  ;   C02       ; % mole
#Hydrogen H2;   H         ; % mole  -> max: 0,03%

#estos no están, se podrían agregar en un futuro
#n-Hexane   ;   C6H14     ; % mole
#n-Heptane  ;   C7H16     ; % mole
#n-Octane   ;   C8H18     ; % mole
#n-Nonane   ;   C9H20     ; % mole
#n-Decane   ;   C10H22    ; % mole
#Hydrogen H2;   H         ; % mole  -> max: 0,03%
#Sulfuro H2 ;   H2S       ; ppmv    -> max: 6 ppmv
#Monox Carb ;   CO        ; % mole
#Oxygeno    ;   O2        ; % mole
#Siloxane   ;   SI        ; % mole  -> max: 0,0003%
#Sulfur     ;   S2        ; % peso  -> max: 0,001% (10ppm weight)
# Como resultado tendriamos tres parametros:
# Lower Heating Value [BTU/lbm] (mas brigido)
# Widgets: Label(root, text, fg), Entry(), Radiobutton(root, text="", value=1, variable= ), Button()
# ej: Button(root, text="BLABLA").grid(row=0,column=0) ó .place(x=0,y=0) ó .pack(side=RIGHT/LEFT)